# DC-DCコンバータでのパラメータ最適化

### 最適化の流れ

1. パラメータ設定
L, C, f_sw, Rloadの範囲を設定

2. データ生成
SPICEでパラメータスイープ (PySpice, ngspice)

3. AI学習
特性モデルを学習 scikit-learn, PyTorch

4. 最適化
効率最大・リップル最小化 (Optuna)


6. 検証・可視化
結果をSPICEで検証(matplotlib)


In [1]:
import numpy as np
import subprocess
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import optuna
import numpy as np
import matplotlib.pyplot as plt

/Users/enty/.local/share/virtualenvs/Circuit_Design-i0K2t5yT/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
import subprocess
import pandas as pd

L_values = np.linspace(1e-6, 100e-6, 10)
C_values = np.linspace(1e-6, 100e-6, 10)

results = []

for L in L_values:
    for C in C_values:
        # SPICEファイルの書き換え
        with open("template.cir", "r") as f:
            netlist = f.read().replace("LVAL", str(L)).replace("CVAL", str(C))
        with open("run.cir", "w") as f:
            f.write(netlist)

        # SPICEを実行
        subprocess.run(["ngspice", "-b", "run.cir", "-o", "result.log"])
        
        # ログから効率を抽出（仮）
        efficiency = parse_efficiency("result.log")
        results.append([L, C, efficiency])

# データ保存
pd.DataFrame(results, columns=["L", "C", "efficiency"]).to_csv("data.csv", index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

data = pd.read_csv("data.csv")
X = data[["L", "C"]]
y = data["efficiency"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500)
model.fit(X_scaled, y)

# 精度確認
pred = model.predict(X_scaled)
plt.scatter(y, pred)
plt.xlabel("True Efficiency")
plt.ylabel("Predicted Efficiency")
plt.show()

In [ ]:
import optuna
import numpy as np

def objective(trial):
    L = trial.suggest_loguniform('L', 1e-6, 100e-6)
    C = trial.suggest_loguniform('C', 1e-6, 100e-6)
    
    X_test = scaler.transform([[L, C]])
    eff_pred = model.predict(X_test)[0]
    return -eff_pred  # 最大化したいので符号反転

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("最適値:", study.best_params)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

L_range = np.linspace(1e-6, 100e-6, 50)
C_range = np.linspace(1e-6, 100e-6, 50)
Z = np.zeros((50, 50))

for i, L in enumerate(L_range):
    for j, C in enumerate(C_range):
        X = scaler.transform([[L, C]])
        Z[i, j] = model.predict(X)

plt.imshow(Z, origin='lower', extent=[1e-6, 100e-6, 1e-6, 100e-6], aspect='auto')
plt.xlabel("L [H]")
plt.ylabel("C [F]")
plt.title("Predicted Efficiency Map")
plt.colorbar(label="Efficiency")
plt.show()